In [3]:
import spotipy
import pandas as pd
import requests
from spotipy.oauth2 import SpotifyClientCredentials
import base64
import csv
import sqlite3
from time import sleep


In [88]:
client_id = "5c45b3b09a1b424faae5b522d584a013"
client_secret = "6b011551459a43d2ac6ab49fec394dc4"

In [89]:
audio_features = [
  'danceability',
  'energy',
  'key',
  'loudness',
  'mode',
  'speechiness',
  'acousticness',
  'instrumentalness',
  'liveness',
  'valence',
  'tempo',
  'duration_ms',
  'time_signature'
  ]

In [90]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

# encode client_id and client_secret in base64
client_credentials = f"{client_id}:{client_secret}".encode("ascii")
base64_credentials = base64.b64encode(client_credentials).decode("ascii")

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [91]:
# make request for access token
url = "https://accounts.spotify.com/api/token"
response = requests.post(
    url,
    headers={
        "Authorization": f"Basic {base64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data={"grant_type": "client_credentials"}
)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print(f"Access Token: {access_token}")
else:
    print(f"Error: {response.text}")

Access Token: BQBKIIbsEaw1GFDjTX2tLEC4bEu3fooGDrYsPqyFaoXa0SElg3XaeaSNA2811FhtHOuhO1DaOdIvdAaNxvlQg21O2CGIWGcwi8YeLoQob726WR_RYODb


In [92]:
url = "https://api.spotify.com/v1/markets"

response = requests.get(
    url,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
)

if response.status_code == 200:
    data = response.json()
    markets_list = data["markets"]
    print(markets_list)
else:
    print("Erreur lors de la récupération des marchés : ", response.text)

['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', 'NA', 'NE', 'NG', 'NI', 'NL', 'NO', 'NP', 'NR', 'NZ', 'OM', 'PA', 'PE', 'PG', 'PH', 'PK', 'PL', 'PS', 'PT', 'PW', 'PY', 'QA', 'RO', 'RS', 'RW', 'SA', 'SB', 'SC', 'SE', 'SG', 'SI', 'SK', 'SL', 'SM', 'SN', 'SR', 'ST', 'SV', 'SZ', 'TD', 'TG', 'TH', 'TJ', 'TL', 'TN', 'TO', 'TR', 'TT

In [30]:
dfs = []
c=0
for genre in sp.recommendation_genre_seeds()['genres']:
    sleep(1)
    c+=1
    print(genre, c)
    for offset in range(0,1000,50):
        results = sp.search(q=f'year:2022 and genre:{genre}', type='track', limit=50, offset=offset)
        df = pd.DataFrame.from_dict(results['tracks']['items'])
        df['genre'] = genre
        dfs.append(df)
df_items = pd.concat(dfs)
df_items.to_csv('Spotify.csv', index=False)

In [99]:
df_items['album'] = df_items['album'].apply(lambda album : album['album_type'])
df_items['available_markets'] = df_items['available_markets'].apply(lambda x : len(x))
df_items = df_items[['id','popularity','available_markets','genre']]
df_items['audio_features'] = df_items['id'].apply(lambda id : sp.audio_features(id))
df_items = df_items[~df_items['audio_features'].isin([[None]])]

for feature in audio_features:
    df_items[feature] = df_items['audio_features'].apply(lambda x: x[0][feature])
df_items = df_items.drop(columns=['audio_features','id'])
df_items = df_items.drop_duplicates()
df_items.to_csv('spotify.csv', index=False)
df_items['id']=df_items.index

/tmp/ipykernel_20362/3646842874.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['audio_features'] = df_items['id'].apply(lambda id : sp.audio_features(id))


In [4]:
# df_items = pd.read_csv('spotify_best.csv')
# df_items['id'] = df_items.index
connexion = sqlite3.connect('Spotify.sqlite')
curseur = connexion.cursor()
df_items.to_sql("TRACKS", connexion, if_exists="replace")
connexion.commit()
connexion.close()

NameError: name 'df_items' is not defined

In [5]:
connexion = sqlite3.connect('Spotify.sqlite')
curseur = connexion.cursor()
df = pd.read_sql_query("SELECT * FROM TRACKS;", connexion)
connexion.commit()
connexion.close()

In [177]:
df['category'] = df['popularity'].apply(lambda p : int(p//25))
df.to_csv('spotify.csv',index=False)

/tmp/ipykernel_20362/370939326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = df['popularity'].apply(lambda p : int(p//25))
